In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier,ExtraTreesRegressor , RandomForestRegressor
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
import pickle
from sklearn.neural_network import MLPClassifier

C:\Users\master\Anaconda3\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
C:\Users\master\Anaconda3\lib\site-packages\dask\dataframe\utils.py:13: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
C:\Users\master\Anaconda3\lib\site-packages\distributed\config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [2]:
df = pd.read_csv('submission files/train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,853.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.635604,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.167261,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,21.000000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [4]:
print(df.info())
df['Age'] = df['Age'].fillna(df['Age'].mean())
print(df.info())
labels = ['child','adult', 'old']
df['age_group'] = pd.cut(df['Age'], bins=3, labels=labels)
df['age_group']= df['age_group'].fillna('adult')
df.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          853 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Sur

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
age_group        0
dtype: int64

In [5]:
labels = ['cheap', 'normal', 'expensive']
df['Fare_group'] = pd.cut(df['Fare'], bins=3, labels=labels)
df['Fare_group']= df['Fare_group'].fillna('cheap')
print(df.isnull().sum())
df

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
age_group        0
Fare_group       0
dtype: int64


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,age_group,Fare_group
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,child,cheap
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,adult,cheap
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,child,cheap
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,adult,cheap
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,adult,cheap
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,adult,cheap
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,child,cheap
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,7.0,1,2,W./C. 6607,23.4500,NaN,S,child,cheap
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,child,cheap


In [6]:
df.drop(columns= ['PassengerId', 'Name', 'Cabin', 'Embarked', 'Ticket','Age', 'Fare'], inplace=True)
df

,Survived,Pclass,Sex,SibSp,Parch,age_group,Fare_group
0,0,3,male,1,0,child,cheap
1,1,1,female,1,0,adult,cheap
2,1,3,female,0,0,child,cheap
3,1,1,female,1,0,adult,cheap
4,0,3,male,0,0,adult,cheap
...,...,...,...,...,...,...,...
886,0,2,male,0,0,adult,cheap
887,1,1,female,0,0,child,cheap
888,0,3,female,1,2,child,cheap
889,1,1,male,0,0,child,cheap


In [7]:
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
df

,Survived,Pclass,Sex,SibSp,Parch,age_group,Fare_group,FamilySize
0,0,3,male,1,0,child,cheap,2
1,1,1,female,1,0,adult,cheap,2
2,1,3,female,0,0,child,cheap,1
3,1,1,female,1,0,adult,cheap,2
4,0,3,male,0,0,adult,cheap,1
...,...,...,...,...,...,...,...,...
886,0,2,male,0,0,adult,cheap,1
887,1,1,female,0,0,child,cheap,1
888,0,3,female,1,2,child,cheap,4
889,1,1,male,0,0,child,cheap,1


In [8]:
df.loc[df['FamilySize'] == 1, 'IsAlone'] = 1
df['IsAlone']= df['IsAlone'].fillna(0)
df

,Survived,Pclass,Sex,SibSp,Parch,age_group,Fare_group,FamilySize,IsAlone
0,0,3,male,1,0,child,cheap,2,0.0
1,1,1,female,1,0,adult,cheap,2,0.0
2,1,3,female,0,0,child,cheap,1,1.0
3,1,1,female,1,0,adult,cheap,2,0.0
4,0,3,male,0,0,adult,cheap,1,1.0
...,...,...,...,...,...,...,...,...,...
886,0,2,male,0,0,adult,cheap,1,1.0
887,1,1,female,0,0,child,cheap,1,1.0
888,0,3,female,1,2,child,cheap,4,0.0
889,1,1,male,0,0,child,cheap,1,1.0


In [9]:
df.drop(columns=['SibSp', 'Parch'], inplace=True)
df

,Survived,Pclass,Sex,age_group,Fare_group,FamilySize,IsAlone
0,0,3,male,child,cheap,2,0.0
1,1,1,female,adult,cheap,2,0.0
2,1,3,female,child,cheap,1,1.0
3,1,1,female,adult,cheap,2,0.0
4,0,3,male,adult,cheap,1,1.0
...,...,...,...,...,...,...,...
886,0,2,male,adult,cheap,1,1.0
887,1,1,female,child,cheap,1,1.0
888,0,3,female,child,cheap,4,0.0
889,1,1,male,child,cheap,1,1.0


In [10]:
y = df.iloc[:, 0].values
X = df.iloc[:,1:].values
X

array([[3, 'male', 'child', 'cheap', 2, 0.0],
       [1, 'female', 'adult', 'cheap', 2, 0.0],
       [3, 'female', 'child', 'cheap', 1, 1.0],
       ...,
       [3, 'female', 'child', 'cheap', 4, 0.0],
       [1, 'male', 'child', 'cheap', 1, 1.0],
       [3, 'male', 'adult', 'cheap', 1, 1.0]], dtype=object)

In [11]:
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1,2,3])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[0.0 1.0 0.0 ... 3 2 0.0]
 [1.0 0.0 1.0 ... 1 2 0.0]
 [1.0 0.0 0.0 ... 3 1 1.0]
 ...
 [1.0 0.0 0.0 ... 3 4 0.0]
 [0.0 1.0 0.0 ... 1 1 1.0]
 [0.0 1.0 1.0 ... 3 1 1.0]]


In [120]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 11)
print(X_train)
print(X_test)
print(y_train)
print(y_test)

[[1.0 0.0 1.0 ... 3 1 1.0]
 [0.0 1.0 1.0 ... 3 1 1.0]
 [1.0 0.0 0.0 ... 3 1 1.0]
 ...
 [0.0 1.0 0.0 ... 3 1 1.0]
 [0.0 1.0 0.0 ... 3 1 1.0]
 [0.0 1.0 0.0 ... 3 1 1.0]]
[[1.0 0.0 1.0 ... 3 2 0.0]
 [0.0 1.0 1.0 ... 3 1 1.0]
 [0.0 1.0 0.0 ... 3 1 1.0]
 ...
 [1.0 0.0 1.0 ... 3 2 0.0]
 [1.0 0.0 1.0 ... 1 2 0.0]
 [0.0 1.0 1.0 ... 2 2 0.0]]
[0 1 1 0 0 0 0 0 0 1 0 1 1 0 1 0 0 1 0 1 1 0 1 0 1 0 1 0 1 1 0 0 1 1 0 1 0
 1 1 0 1 0 1 1 0 0 1 0 1 0 0 0 0 0 0 1 0 1 0 0 0 1 1 1 1 0 0 1 0 1 0 0 0 0
 0 0 0 1 0 1 0 1 1 0 0 1 0 1 1 0 0 1 0 1 1 0 1 1 1 1 1 0 0 0 0 0 0 0 1 0 1
 0 0 1 1 0 0 1 0 0 1 1 1 0 1 1 1 1 0 0 0 0 0 1 1 0 1 1 1 1 0 0 1 0 1 0 1 0
 0 0 1 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 1 0 1 0 0 1 0 1 1 0 1 1 0 1 1
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 0 0 0 0 1 0 0 1 0 1 0 0 0 1
 0 0 0 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 1 1 0 1 0 0 1 0 0 1 1
 0 1 1 0 0 1 0 1 1 0 0 0 1 1 0 0 1 0 1 1 0 1 1 0 0 1 0 0 0 1 1 1 1 0 1 1 1
 0 1 0 0 0 1 1 0 1 0 1 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0 0 0 0 0 0 0

In [121]:
sc = StandardScaler()
X_train[:, 2:] = sc.fit_transform(X_train[:, 2:])
X_test[:, 2:] = sc.transform(X_test[:, 2:])
print(X_train)
print(X_test)

[[1.0 0.0 0.9951961291440299 ... 0.8377160158809513 -0.5501115895340078
  0.7996709849747765]
 [0.0 1.0 0.9951961291440299 ... 0.8377160158809513 -0.5501115895340078
  0.7996709849747765]
 [1.0 0.0 -1.004827059426069 ... 0.8377160158809513 -0.5501115895340078
  0.7996709849747765]
 ...
 [0.0 1.0 -1.004827059426069 ... 0.8377160158809513 -0.5501115895340078
  0.7996709849747765]
 [0.0 1.0 -1.004827059426069 ... 0.8377160158809513 -0.5501115895340078
  0.7996709849747765]
 [0.0 1.0 -1.004827059426069 ... 0.8377160158809513 -0.5501115895340078
  0.7996709849747765]]
[[1.0 0.0 0.9951961291440299 ... 0.8377160158809513 0.0629823644546985
  -1.2505142974914203]
 [0.0 1.0 0.9951961291440299 ... 0.8377160158809513 -0.5501115895340078
  0.7996709849747765]
 [0.0 1.0 -1.004827059426069 ... 0.8377160158809513 -0.5501115895340078
  0.7996709849747765]
 ...
 [1.0 0.0 0.9951961291440299 ... 0.8377160158809513 0.0629823644546985
  -1.2505142974914203]
 [1.0 0.0 0.9951961291440299 ... -1.5291641559731

In [ ]:
classifierlg = LogisticRegression(random_state = 1)
classifierlg.fit(X_train, y_train)

In [ ]:
y_pred = classifierlg.predict(X_test)
print(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

In [ ]:
filename = 'models/LogisticRegression_model2079.sav'
pickle.dump(classifierlg, open(filename, 'wb'))

In [ ]:
classifierknn = KNeighborsClassifier(n_neighbors = 3, metric = 'minkowski', p = 2)
classifierknn.fit(X_train, y_train)

In [ ]:
y_pred = classifierknn.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

In [ ]:
filename = 'models/KNeighborsClassifier_model2073.sav'
pickle.dump(classifierknn, open(filename, 'wb'))

In [ ]:
classifiersvc = SVC(C = 0.5, kernel = 'linear', random_state = 1)
classifiersvc.fit(X_train, y_train)

In [ ]:
y_pred = classifiersvc.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

In [ ]:
filename = 'models/SVC_model2078.sav'
pickle.dump(classifiersvc, open(filename, 'wb'))

In [ ]:
classifierdct = DecisionTreeClassifier(criterion = 'entropy', random_state = 1)
classifierdct.fit(X_train, y_train)

In [ ]:
y_pred = classifierdct.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

In [ ]:
filename = 'models/DecisionTreeClassifier_model2078.sav'
pickle.dump(classifierdct, open(filename, 'wb'))

In [ ]:
classifierf = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 5)
classifierf.fit(X_train, y_train)

In [ ]:
y_pred = classifierf.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

In [ ]:
filename = 'models/RandomForestClassifier_model2078.sav'
pickle.dump(classifierf, open(filename, 'wb'))

In [ ]:
classifierxgb = XGBClassifier()
classifierxgb.fit(X_train, y_train)

In [ ]:
y_pred = classifierxgb.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

In [ ]:
filename = 'models/XGBClassifier_model2078.sav'
pickle.dump(classifierxgb, open(filename, 'wb'))

In [126]:
model = MLPClassifier(hidden_layer_sizes=(64,32,16,2),
                      solver='adam',
                      alpha=0.0001,
                      learning_rate_init = 0.001
                      activation='identity',
                     learning_rate='adaptive',
                     batch_size=128,verbose=True, early_stopping=True)

SyntaxError: invalid syntax (<ipython-input-126-9c53564dee7a>, line 5)

In [127]:
model.fit(X_train,y_train)

Iteration 1, loss = 1.34241377
Validation score: 0.396825
Iteration 2, loss = 0.73154248
Validation score: 0.682540
Iteration 3, loss = 0.56343335
Validation score: 0.777778
Iteration 4, loss = 0.55596137
Validation score: 0.793651
Iteration 5, loss = 0.51678365
Validation score: 0.793651
Iteration 6, loss = 0.47831126
Validation score: 0.809524
Iteration 7, loss = 0.46760322
Validation score: 0.793651
Iteration 8, loss = 0.47403562
Validation score: 0.793651
Iteration 9, loss = 0.47546673
Validation score: 0.793651
Iteration 10, loss = 0.47069842
Validation score: 0.793651
Iteration 11, loss = 0.47056187
Validation score: 0.809524
Iteration 12, loss = 0.46885584
Validation score: 0.825397
Iteration 13, loss = 0.46879678
Validation score: 0.809524
Iteration 14, loss = 0.46736383
Validation score: 0.825397
Iteration 15, loss = 0.46785277
Validation score: 0.825397
Iteration 16, loss = 0.46890396
Validation score: 0.825397
Iteration 17, loss = 0.46760515
Validation score: 0.825397
Iterat

MLPClassifier(activation='identity', alpha=0.001, batch_size=128,
              early_stopping=True, hidden_layer_sizes=(64, 32, 16, 2),
              learning_rate='adaptive', verbose=True)

In [128]:
predictions = model.predict(X_test)
print('Accuracy score ',accuracy_score(y_test,predictions))

Accuracy score  0.8544776119402985


In [129]:
filename = 'MLPClassifier30854.sav'
pickle.dump(model, open(filename, 'wb'))